# Main

In [1]:
import pandas as pd
import math
from scipy import interpolate

In [2]:
# Definition of constants and variables

# Some common physical constants, mainly used to calculate `maxFlux`
## (the upper bound of energy flux received by a planet that allows it
## to be habitable according to Pierrehumbert (2015))

BIGG       = 6.67428e-11       # Gravitational constant
PI         = 3.1415926535
A          = 0.7344            # Pierrehumbert's Constant
SB         = 5.670373e-8       # Stefan-Boltzmann Constant
LH2O       = 2.425e6           # Latent Heat Capacity of Water
RGAS       = 461.5             # Universal Gas Constant
PLINE      = 1e4               
PREF       = 610.616           # Reference Pressure
TREF       = 273.13            # Reference Temperature
K0         = 0.055             # A constant in Runaway Greenhouse calculation


# Boundaries for albedo values:
ALBMINELSE = 0.05              # General lower bound
ALBMAXELSE = 0.8               # General upper bound
ALBMING    = 0.25              # Lower bound for planets orbiting G-type stars
ALBMAXM    = 0.35              # Upper bound for planets orbiting M-type stars


# The counterpart to maxFlux
## (unlike maxFlux, MINFLUX is a constant that does not depend on a planet's properties)
MINFLUX    = 67

# Definitions of units of measurement,
## mainly used to convert [exoplanet.org](exoplanet.org) data into SI units:
MEARTH     = 5.972186e24       # Earth mass in kilograms
REARTH     = 6378100           # Earth's radius in meters
S0         = 1362              # Solar constant in watts per square meter
MSUN       = 1.988416e30       # Solar mass in kilograms
RSUN       = 6.957e8           # Solar radius in meters
LSUN       = 3.828e26          # Solar luminosity in watts
RJUP       = 7.1492e7          # Jovian radius in meters
MJUP       = 1.8982e27         # Jovian mass in kilograms
AU         = 1.496e11          # The astronomical unit in meters

In [3]:
# Definition of functions

# 1) A function to calculate the probability distribution of orbital eccentricity
def pofe(ecc,mu,sigma):
    return ((sigma*math.sqrt(2*math.pi))**(-1))*math.exp(-(((ecc-mu)**2)/(2*sigma**2)))/1000

# 2) A function to calculate the probability of a planet's terrestriality
def fp_ter(mPlanet,rPlanet,exoName):
    # Convert the unit to Earth's masses and radii
    mPlanet = mPlanet/MEARTH 
    rPlanet = rPlanet/REARTH
    
    # Calculate mu1
    ## Initialize mu1 value to 0
    mu1 = 0.0       
    
    # Initialize temporary variables to hold a mass/radius value
    ## from the (i-1)th row of the ZS table
    mZSimin1 = 0
    rZSimin1 = 0

    # This block iterates through the the 'M-Pure-MgSiO3' column
    #to find the bracket that contains mPlanet value
    for i in rowNum:
        # Initialize temporary variables to hold a mass/radius value
        #from the i-th row of the ZS table
        mZSi = zs.loc[i, "M-PureMgSiO3"]
        rZSi = zs.loc[i, "R-PureMgSiO3"]
        
        # Comparing mPlanet to the current value of mZSi
        if mPlanet == mZSi:
            mu1 = rZSi
            break                          
        elif mPlanet > mZSi:               
            mZSimin1 = mZSi                
            rZSimin1 = rZSi                  
        else: # if mPlanet < mZSi --> we have found the correct bracket
            f = interpolate.interp1d(zs.loc[(i-1):(i), "M-PureMgSiO3"], zs.loc[(i-1):(i), "R-PureMgSiO3"], kind='linear', assume_sorted=True)
            mu1 = f(mPlanet)
            break

    # Calculate mu2
    mu2 = 0.0
    mZSimin1 = 0
    rZSimin1 = 0
    for i in rowNum:
        mZSi = zs.loc[i, "M-MgSiO3-H2O-5050"]
        rZSi = zs.loc[i, "R-MgSiO3-H2O-5050"]
        if mPlanet == mZSi:
            mu2 = rZSi
            break
        elif mPlanet > mZSi:
            mZSimin1 = mZSi
            rZSimin1 = rZSi
        else: 
            f = interpolate.interp1d(zs.loc[(i-1):(i), "M-MgSiO3-H2O-5050"], zs.loc[(i-1):(i), "R-MgSiO3-H2O-5050"], kind='linear', assume_sorted=True)
            mu2 = f(mPlanet)
            break

    # Calculate sigma1
    sigma1 = (mu2-mu1)/3
    
    # Calculate the terrestrial probability
    p_ter = 0
    if rPlanet <= mu1:
        p_ter = 1
    elif rPlanet >= mu2:
        p_ter = 0
    else: # uses a pseudo-gaussian function
        p_ter = math.exp(-(0.5)*((rPlanet-mu1)/sigma1)**2)
    return p_ter

In [4]:
# Data input
## Import exoplanet data from a CSV file into a pandas dataframe
exo = pd.read_csv (r'nasa_exoplanets.csv', low_memory=False)

# Set the column with the header NAME to be used as an index to identify row 
exo = exo.set_index("pl_name", drop = False)

# Extract names of planets as a list (to be used as a calling list)
exoList = pd.DataFrame(exo, columns=['pl_name'])
exoList = exoList['pl_name'].values.tolist()

In [5]:
# Zeng-Sasselov boundaries input

## Import CSV of Zeng & Sasselov boundaries
zs = pd.read_csv (r'zsboundaries.csv')

## Set index using the RowNum column
zs = zs.set_index("RowNum", drop = False)

## Extract the column "RowNum" as a list (to be used as a calling list)
rowNum = pd.DataFrame(zs, columns=['RowNum'])
rowNum = rowNum['RowNum'].values.tolist()

In [6]:
# Main subroutine to determine the habitability index value

habIndex = []
habIndexWithName = []
habIndexNotZero = []
i = 1

for exoName in exoList:
    print(exoName)
    # Extract data of individual planets
    
    # HOST STAR PROPERTIES
    # Stellar radius (in solar radii)
    rStar = exo.loc[exoName, "st_rad"]
    ## Convert to SI
    rStar = rStar*RSUN
    
    # Stellar temperature (in Kelvin)
    teffStar = exo.loc[exoName, "st_teff"]
    
    # Stellar luminosity
    luminosity = 4*math.pi*rStar*rStar*SB*teffStar**4
    
    # PLANET PROPERTIES   
    # Planetary radius (in Jovian radii)
    rPlanet = exo.loc[exoName, "pl_radj"]
    ## If R is not available, calculate it from  transit depth
    if math.isnan(rPlanet) == 1:
        depth = exo.loc[exoName, "pl_trandep"]
        if math.isnan(depth) == 1:
            continue
        rPlanet = math.sqrt(depth)*rStar
    ## Convert to SI
    rPlanet = rPlanet*RJUP
    
    # Planetary mass (in Jovian masses)
    mPlanet = exo.loc[exoName, "pl_bmassj"] 
    ## If MASS is not available, calculate it from a common scaling law
    ### from the original HITE
    if math.isnan(mPlanet) == 1:
        if rPlanet/REARTH <= 1:
            mPlanet = ((rPlanet/REARTH)**3.268)*MEARTH
        elif rPlanet/REARTH > 1:
            mPlanet = ((rPlanet/REARTH)**3.65)*MEARTH
    ## Convert to SI
    mPlanet = mPlanet*MJUP
    
    # Surface planet gravity (in SI)
    surfGrav = BIGG*mPlanet/(rPlanet**2)    
    
    # ORBITAL PROPERTIES
    
    # Orbital eccentricity
    ecc = exo.loc[exoName, "pl_orbeccen"]

    # Measurement uncertainty of orbital eccentricity    
    ## Upper bound (relative from E)
    eccUpRel = exo.loc[exoName, "pl_orbeccenerr1"]
    ### If measurement uncertainty is not available, assign it as 0.01
    if math.isnan(eccUpRel) == 1:
        eccUpRel = 0.01
    ### Upper bound (absolute)
    eccUpper = ecc + eccUpRel
    
    ## Lower bound (relative from E)
    eccLowRel = exo.loc[exoName, "pl_orbeccenerr2"]
    ### If measurement uncertainty is not available, assign it as 0.01
    if math.isnan(eccLowRel) == 1:
        eccLowRel = 0.01
    ###Lower bound (absolute)
    eccLower = ecc - eccLowRel
    
    # Orbital semi-major axis (in AU)
    semiAxis = exo.loc[exoName, "pl_orbsmax"]
    ## Convert to SI
    semiAxis = semiAxis*AU
    
    
    # Calculate the upper and lower bounds of F_OLR [...]
    ## that would allow for surface liquid water to exist
    pStar = PREF*math.exp(LH2O/(RGAS*TREF))
    # Upper bound: maximum F_OLR
    maxFlux = A*SB*(LH2O/(RGAS*math.log(pStar*math.sqrt(K0/(2*PLINE*surfGrav)))))**4
    # Lower bound: minimum F_OLR is the constant MINFLUX
    minFlux = MINFLUX
    
    # Probability of the planet being terrestrial
    p_ter = fp_ter(mPlanet,rPlanet,exoName)
        
    
    # Albedo (new)
    ## Boundaries
    albMin = ALBMINELSE
    albMax = ALBMAXELSE
    ## Special conditions
    ### For planets with M-type host star
    if teffStar >= 2300 and teffStar <=3800:
        albMax = ALBMAXM
    ### For planets with G-type host star
    elif teffStar >= 5370 and teffStar <=5980:
        albMin = ALBMING
        
        
    
    # Calculate F_OLR
    ## Albedo increments
    da = 0.01
    ## Eccentricity increments
    de = 0.01
    ## Sum of pofe (probability of eccentricity);
    ### (is used to normalize the index value, later)
    ### Initialized to 0
    pofeSum = 0
    ### Sum of how many instances of F_OLR meets the requirements for
    #### the planet to have surface liquid water. Each instances will then be
    #### multiplied by the probability of its eccentricity (pofe)
    ### Initialized to 0
    habFact = 0
    ### Incoming stellar radiation (instellation)
    flux0 = luminosity/(16*math.pi*semiAxis*semiAxis)

    # Calculate the habitability index
    ## Iterate through the albedo & eccentricity 2D matrix
    a = albMin
    while a < albMax:
        e = eccLower
        while e < eccUpper:
            flux = flux0*(1-a)/math.sqrt(1-e*e)
            pofeSum = pofeSum + pofe(e, ecc, eccUpRel)
            if flux < maxFlux and flux > MINFLUX:
                habFact = habFact + pofe(e, ecc, eccUpRel)
            e = e + de
        a = a + da   
    
    if ecc > 0.8:
        H = 0.0
    elif pofeSum != 0:
        H = (habFact/pofeSum)*p_ter
    else: # in the case of error; might be better to replace this with a throw exception statement
        H = 0.0
    
    habIndex.append(H)
    habIndexWithName.extend([exoName, ",", H])
    
    if H > 0:
        habIndexNotZero.extend([exoName+ ": "+str(H)])
        
    print(i)
    i = i +1

11 Com b
1
11 UMi b
2
14 And b
3
14 Her b
4
16 Cyg B b
5
17 Sco b
6
18 Del b
7
1RXS J160929.1-210524 b
8
24 Boo b
9
24 Sex b
10
24 Sex c
11
2M0437 b
12
2MASS J01033563-5515561 AB b
13
2MASS J01225093-2439505 b
14
2MASS J02192210-3925225 b
15
2MASS J04414489+2301513 b
16
2MASS J12073346-3932539 b
17
2MASS J19383260+4603591 b
18
2MASS J22362452+4751425 b
19
30 Ari B b
20
4 UMa b
21
42 Dra b
22
47 UMa b
23
47 UMa c
24
47 UMa d
25
51 Eri b
26
51 Peg b
27
55 Cnc b
28
55 Cnc c
29
55 Cnc d
30
55 Cnc e
31
55 Cnc f
32
6 Lyn b
33
61 Vir b
34
61 Vir c
35
61 Vir d
36
7 CMa b
37
7 CMa c
38
70 Vir b
39
75 Cet b
40
8 UMi b
41
81 Cet b
42
91 Aqr b
43
AB Aur b
44
AB Pic b
45
AU Mic b
46
AU Mic c
47
BD+03 2562 b
48
BD+14 4559 b
49
BD+15 2375 b
50
BD+15 2940 b
51
BD+20 2457 b
52
BD+20 2457 c
53
BD+20 274 b
54
BD+20 594 b
55
BD+45 564 b
56
BD+48 738 b
57
BD+48 740 b
58
BD+49 828 b
59
BD+55 362 b
60
BD+60 1417 b
61
BD+63 1405 b
62
BD-00 4475 b
63
BD-06 1339 b
64
BD-06 1339 c
65
BD-08 2823 b
66
BD-08 2823 c

In [7]:
for i in habIndexNotZero:
    print(i)

GJ 1061 d: 1.0
GJ 273 b: 0.15112609881783914
K2-3 c: 0.00046109522713008787
K2-3 d: 0.21409345359060547
K2-72 e: 0.41106434804222747
Kepler-1185 b: 0.07271643574261376
Kepler-1229 b: 0.9305885753725621
Kepler-1389 b: 0.011200170972980903
Kepler-1410 b: 0.019340872963142207
Kepler-1450 b: 0.009610694124363656
Kepler-1459 b: 0.07958379064276017
Kepler-1544 b: 0.03315578222252952
Kepler-1599 b: 0.004003310729984849
Kepler-1605 b: 0.10666666666666665
Kepler-1649 c: 0.13333333333333333
Kepler-1652 b: 0.28272655826039955
Kepler-186 f: 0.6666666666666661
Kepler-22 b: 1.0
Kepler-220 e: 0.05332538621125
Kepler-283 c: 0.022601240173071403
Kepler-296 f: 0.025450250421797632
Kepler-331 d: 0.026935926216663714
Kepler-367 c: 0.09333333333333332
Kepler-395 c: 0.13333333333333336
Kepler-441 b: 0.016467101777025192
Kepler-442 b: 0.9163113199860241
Kepler-452 b: 0.2551170189584821
Kepler-538 b: 0.0359360732459901
Kepler-55 b: 0.06666666666666667
Kepler-55 c: 0.36000000000000004
Kepler-62 e: 1.0
Kepler-6

In [8]:
# Append the result (planet's name and index value) to a .txt file in the same folder 
with open('out_nasadb.txt','w') as f:
    i = 1
    for a in habIndexWithName:
        if i == 3:
            print(a, file=f)
            i = 1
        else:
            print(a, file=f, end="")
            i += 1

In [9]:
5233

5233